In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
import os
from tqdm import *
from sklearn.metrics import *
import warnings 
warnings.simplefilter('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactive = "all"

In [2]:
%%time
payment_history = pd.read_csv('payment_history.csv')
client_data = pd.read_csv('client_data.csv')
policy_df = pd.read_csv('policy_data.csv')
train_data = pd.read_csv('Trainset.csv')

Wall time: 1.87 s


In [3]:
client_data.sample(4, random_state=0)

,Policy ID,NPH_TITLE,NPH_LASTNAME,NPH_SEX,NPH_BIRTHDATE,NAD_ADDRESS1,NAD_ADDRESS2
86913,PID_EK8DGXF,Ms.,NPH_LASTNAME_P2TIXC7,F,1999,ADDRESS1_WSRXQ43,ADDRESS2_AIIHWOT
48501,PID_8VJ70ZJ,Ms.,NaN,F,2003,ADDRESS1_WSRXQ43,ADDRESS2_AIIHWOT
117158,PID_VJLU1FW,Miss.,NPH_LASTNAME_3U48T5E,F,1988,NaN,NaN
43259,PID_R95OVMI,Mr.,NaN,M,2007,ADDRESS1_WSRXQ43,ADDRESS2_AIIHWOT


In [4]:
test_set =train_data.loc[train_data["Lapse"]=="?"]

train_set=train_data.loc[train_data["Lapse"] !="?"]

# Creating Train Data

In [5]:
first_mrg = pd.merge(policy_df, train_set, on="Policy ID",how='left')
first_mrg['Lapse'] = first_mrg['Lapse'].replace(np.nan,0)
first_mrg['Lapse'] = first_mrg['Lapse'].replace('1',1)
first_mrg.tail()

,Policy ID,NP2_EFFECTDATE,PPR_PRODCD,NPR_PREMIUM,NPH_LASTNAME,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,NLO_AMOUNT,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY,Lapse,Lapse Year,Customer Info Exist,Has Payment History,Policy Info Exist
282810,PID_7P0IXF8,1/5/2017,PPR_PRODCD_KOFUYNN,2109.812579,NPH_LASTNAME_C11UH3D,3,555,74968.903115,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_1OCF2N0,PCL_LOCATCODE_SPQHMX5,OCCUPATION_IKCIDKW,CATEGORY_R821UZV,0,NaN,NaN,NaN,NaN
282811,PID_Q1QLPUQ,1/11/2018,PPR_PRODCD_KOFUYNN,346.027849,NPH_LASTNAME_4PA8HYE,2,222,45513.429325,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_M4MM4QC,PCL_LOCATCODE_SPQHMX5,OCCUPATION_IKCIDKW,CATEGORY_R821UZV,0,NaN,NaN,NaN,NaN
282812,PID_NEDS3A9,1/7/2017,PPR_PRODCD_KOFUYNN,217.975890,NPH_LASTNAME_XF7JXJN,3,333,45513.429325,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_DAP1YRU,PCL_LOCATCODE_0T6GYGX,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,1,2018,0.0,1.0,1.0
282813,PID_U8QKEJK,1/6/2017,PPR_PRODCD_W0F6GK1,5233.874657,NPH_LASTNAME_TE9AFPP,2,222,238857.872515,NLO_TYPE_XAJI0Y6,1075.892081,AAG_AGCODE_L1YVKZO,PCL_LOCATCODE_PEU5TF2,OCCUPATION_IKCIDKW,CATEGORY_R821UZV,0,NaN,NaN,NaN,NaN
282814,PID_5US8QG7,1/9/2019,PPR_PRODCD_W0F6GK1,3990.141182,NPH_LASTNAME_3WBJCBF,1,111,393442.396414,NLO_TYPE_XAJI0Y6,820.226234,AAG_AGCODE_L1YVKZO,PCL_LOCATCODE_PEU5TF2,OCCUPATION_IKCIDKW,CATEGORY_R821UZV,0,NaN,NaN,NaN,NaN


In [6]:
first_mrg.columns

Index(['Policy ID', 'NP2_EFFECTDATE', 'PPR_PRODCD', 'NPR_PREMIUM',
       'NPH_LASTNAME', 'CLF_LIFECD', 'NSP_SUBPROPOSAL', 'NPR_SUMASSURED',
       'NLO_TYPE', 'NLO_AMOUNT', 'AAG_AGCODE', 'PCL_LOCATCODE', 'OCCUPATION',
       'CATEGORY', 'Lapse', 'Lapse Year', 'Customer Info Exist ',
       'Has Payment History', 'Policy Info Exist'],
      dtype='object')

In [7]:
first_mrg['Lapse'].value_counts()

0    225260
1     57555
Name: Lapse, dtype: int64

In [8]:
print('The shape of our train file will be ',first_mrg.shape)

The shape of our train file will be  (282815, 19)


# Creating Test Data

In [9]:
second_mrg  = pd.merge(test_set,policy_df, on="Policy ID",how='left')
second_mrg  = second_mrg.drop_duplicates(subset='Policy ID')
second_mrg.tail()

,Policy ID,Lapse,Lapse Year,Customer Info Exist,Has Payment History,Policy Info Exist,NP2_EFFECTDATE,PPR_PRODCD,NPR_PREMIUM,NPH_LASTNAME,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,NLO_AMOUNT,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY
60854,PID_SS1MY5J,?,?,0,1,1,1/4/2017,PPR_PRODCD_64QNIHM,3187.154301,NPH_LASTNAME_L2UG05W,1,111,93051.091101,NLO_TYPE_XTHV3A3,694.491386,AAG_AGCODE_Z8TCR6I,PCL_LOCATCODE_0T6GYGX,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB
60856,PID_JP7EKSU,?,?,1,1,1,1/12/2018,PPR_PRODCD_64QNIHM,2795.069380,NPH_LASTNAME_XPBWHVN,1,111,143964.463454,NLO_TYPE_XAJI0Y6,471.115025,AAG_AGCODE_LHQRBN4,PCL_LOCATCODE_7VFS3EQ,OCCUPATION_WI64CIY,CATEGORY_GWW4FYB
60858,PID_J1V7RQI,?,?,1,1,1,1/10/2018,PPR_PRODCD_KOFUYNN,694.491386,NPH_LASTNAME_E5S5BF7,2,555,45513.429325,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_AK2UJ2T,PCL_LOCATCODE_O6OBSFL,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB
60870,PID_P2KAC4Y,?,?,0,1,1,1/3/2018,PPR_PRODCD_KOFUYNN,359.045970,NPH_LASTNAME_KBPB6YW,3,444,74968.903115,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_6JA6X8M,PCL_LOCATCODE_7VFS3EQ,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB
60880,PID_8Y9WWHO,?,?,1,1,1,1/11/2018,PPR_PRODCD_W0F6GK1,3607.052915,NPH_LASTNAME_BO119XP,1,111,319834.468881,NLO_TYPE_XAJI0Y6,742.199413,AAG_AGCODE_L1YVKZO,PCL_LOCATCODE_PEU5TF2,OCCUPATION_IKCIDKW,CATEGORY_R821UZV


In [10]:
first_mrg.shape, second_mrg.shape

((282815, 19), (12000, 19))

In [11]:
first_mrg['Customer Info Exist '].isnull().sum()

225260

In [12]:
first_mrg.head(2)

,Policy ID,NP2_EFFECTDATE,PPR_PRODCD,NPR_PREMIUM,NPH_LASTNAME,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,NLO_AMOUNT,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY,Lapse,Lapse Year,Customer Info Exist,Has Payment History,Policy Info Exist
0,PID_EPZDSP8,1/9/2019,PPR_PRODCD_B2KVCE7,265.724174,NPH_LASTNAME_BPN2LEB,2,222,NaN,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_APWOOPE,PCL_LOCATCODE_7SHK7I9,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,0,NaN,NaN,NaN,NaN
1,PID_6M6G9IB,1/8/2018,PPR_PRODCD_64QNIHM,2795.069380,NPH_LASTNAME_U2H3GC6,1,111,213380.713197,NLO_TYPE_XTHV3A3,609.054794,AAG_AGCODE_9Z3FBGA,PCL_LOCATCODE_7VFS3EQ,OCCUPATION_IKCIDKW,CATEGORY_R821UZV,0,NaN,NaN,NaN,NaN


In [13]:
#prepare target
target_y = first_mrg['Lapse'].values

In [14]:
second_mrg=second_mrg.copy()
id_p=second_mrg['Policy ID']

In [15]:
to_drop_cols = ['Policy ID','Lapse Year','NP2_EFFECTDATE','NPH_LASTNAME','Lapse','NLO_AMOUNT']
first_mrg   = first_mrg.drop(to_drop_cols,axis=1)
second_mrg  = second_mrg.drop(to_drop_cols,axis=1)

In [16]:
first_mrg['NPR_SUMASSURED'].fillna(first_mrg['NPR_SUMASSURED'].mean(), inplace=True)
second_mrg['NPR_SUMASSURED'].fillna(second_mrg['NPR_SUMASSURED'].mean(), inplace=True)

In [17]:
first_mrg.shape, second_mrg.shape

((282815, 13), (12000, 13))

In [18]:
first_mrg.columns

Index(['PPR_PRODCD', 'NPR_PREMIUM', 'CLF_LIFECD', 'NSP_SUBPROPOSAL',
       'NPR_SUMASSURED', 'NLO_TYPE', 'AAG_AGCODE', 'PCL_LOCATCODE',
       'OCCUPATION', 'CATEGORY', 'Customer Info Exist ', 'Has Payment History',
       'Policy Info Exist'],
      dtype='object')

In [19]:
first_mrg=first_mrg[['PPR_PRODCD', 'NPR_PREMIUM', 'CLF_LIFECD', 'NSP_SUBPROPOSAL',
       'NPR_SUMASSURED', 'NLO_TYPE', 'AAG_AGCODE', 'PCL_LOCATCODE',
       'OCCUPATION', 'CATEGORY']]
second_mrg=second_mrg[['PPR_PRODCD', 'NPR_PREMIUM', 'CLF_LIFECD', 'NSP_SUBPROPOSAL',
       'NPR_SUMASSURED', 'NLO_TYPE', 'AAG_AGCODE', 'PCL_LOCATCODE',
       'OCCUPATION', 'CATEGORY']]

In [20]:
first_mrg.dtypes

PPR_PRODCD          object
NPR_PREMIUM        float64
CLF_LIFECD           int64
NSP_SUBPROPOSAL      int64
NPR_SUMASSURED     float64
NLO_TYPE            object
AAG_AGCODE          object
PCL_LOCATCODE       object
OCCUPATION          object
CATEGORY            object
dtype: object

In [21]:
first_mrg['PPR_PRODCD'] = first_mrg['PPR_PRODCD'].astype('category').cat.codes
first_mrg['NLO_TYPE'] = first_mrg['NLO_TYPE'].astype('category').cat.codes
first_mrg['AAG_AGCODE'] = first_mrg['AAG_AGCODE'].astype('category').cat.codes
first_mrg['PCL_LOCATCODE'] = first_mrg['PCL_LOCATCODE'].astype('category').cat.codes
first_mrg['OCCUPATION'] = first_mrg['OCCUPATION'].astype('category').cat.codes
first_mrg['CATEGORY'] = first_mrg['CATEGORY'].astype('category').cat.codes

In [22]:
second_mrg['PPR_PRODCD'] = second_mrg['PPR_PRODCD'].astype('category').cat.codes
second_mrg['NLO_TYPE'] = second_mrg['NLO_TYPE'].astype('category').cat.codes
second_mrg['AAG_AGCODE'] = second_mrg['AAG_AGCODE'].astype('category').cat.codes
second_mrg['PCL_LOCATCODE'] = second_mrg['PCL_LOCATCODE'].astype('category').cat.codes
second_mrg['OCCUPATION'] = second_mrg['OCCUPATION'].astype('category').cat.codes
second_mrg['CATEGORY'] = second_mrg['CATEGORY'].astype('category').cat.codes

In [23]:
first_mrg.fillna(230, inplace=True)
second_mrg.fillna(230, inplace=True)

# Modelling

In [24]:
train = first_mrg
test  = second_mrg
train_y = target_y

## Data Summary

In [25]:
train.describe()

,PPR_PRODCD,NPR_PREMIUM,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY
count,282815.000000,282815.000000,282815.000000,282815.000000,2.828150e+05,282815.000000,282815.000000,282815.000000,282815.000000,282815.000000
mean,6.563269,1199.767580,2.565561,314.330135,1.078474e+05,1.700879,302.488277,8.241356,150.310952,2.090101
std,3.850327,2965.959100,1.534623,214.959089,1.728178e+05,1.058978,168.585970,3.777576,36.183737,0.747259
min,0.000000,0.000000,1.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,265.724174,1.000000,111.000000,4.551343e+04,1.000000,169.000000,6.000000,155.000000,2.000000
50%,5.000000,359.045970,3.000000,222.000000,1.078474e+05,1.000000,308.000000,9.000000,155.000000,2.000000
75%,10.000000,1253.156988,3.000000,444.000000,1.078474e+05,3.000000,451.000000,12.000000,155.000000,2.000000
max,16.000000,392777.889967,6.000000,999.000000,4.920032e+07,5.000000,590.000000,14.000000,239.000000,5.000000


In [26]:
test.describe()

,PPR_PRODCD,NPR_PREMIUM,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY
count,12000.000000,12000.000000,12000.000000,12000.00000,1.200000e+04,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000
mean,4.273167,2461.287979,1.724833,194.87750,1.589133e+05,2.558000,254.794750,7.871833,96.477000,2.117417
std,4.098733,3979.417555,1.264087,154.56813,1.267512e+05,1.147205,140.170056,3.722552,30.132465,0.953261
min,0.000000,102.361691,1.000000,1.00000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,368.688310,1.000000,111.00000,8.740046e+04,1.000000,144.000000,6.000000,78.000000,2.000000
50%,1.000000,1951.669846,1.000000,111.00000,1.532721e+05,3.000000,267.000000,9.000000,99.000000,2.000000
75%,9.000000,3187.154301,2.000000,222.00000,1.885467e+05,3.000000,365.250000,11.000000,101.000000,2.000000
max,13.000000,130795.926359,6.000000,999.00000,3.749573e+06,5.000000,502.000000,14.000000,154.000000,5.000000


In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 282815 entries, 0 to 282814
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   PPR_PRODCD       282815 non-null  int8   
 1   NPR_PREMIUM      282815 non-null  float64
 2   CLF_LIFECD       282815 non-null  int64  
 3   NSP_SUBPROPOSAL  282815 non-null  int64  
 4   NPR_SUMASSURED   282815 non-null  float64
 5   NLO_TYPE         282815 non-null  int8   
 6   AAG_AGCODE       282815 non-null  int16  
 7   PCL_LOCATCODE    282815 non-null  int8   
 8   OCCUPATION       282815 non-null  int16  
 9   CATEGORY         282815 non-null  int8   
dtypes: float64(2), int16(2), int64(2), int8(4)
memory usage: 12.9 MB


In [28]:
train.isnull().sum()

PPR_PRODCD         0
NPR_PREMIUM        0
CLF_LIFECD         0
NSP_SUBPROPOSAL    0
NPR_SUMASSURED     0
NLO_TYPE           0
AAG_AGCODE         0
PCL_LOCATCODE      0
OCCUPATION         0
CATEGORY           0
dtype: int64

In [29]:
train.shape

(282815, 10)

In [30]:
target_y.shape

(282815,)

In [31]:
train.nunique()

PPR_PRODCD           17
NPR_PREMIUM        2235
CLF_LIFECD            6
NSP_SUBPROPOSAL     171
NPR_SUMASSURED     1201
NLO_TYPE              6
AAG_AGCODE          591
PCL_LOCATCODE        15
OCCUPATION          240
CATEGORY              6
dtype: int64

In [32]:
train.select_dtypes('object').columns

Index([], dtype='object')

In [33]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_scaled = train
test_scaled = test

In [34]:
pd.DataFrame(train_scaled).head()

,PPR_PRODCD,NPR_PREMIUM,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY
0,5,265.724174,2,222,107847.386640,1,182,5,155,2
1,1,2795.069380,1,111,213380.713197,4,171,6,117,5
2,10,2492.759107,1,111,238857.872515,3,555,12,156,1
3,10,3982.538095,1,111,74968.903115,3,35,13,208,3
4,10,1143.953733,3,555,238857.872515,1,226,1,155,2


In [35]:
pd.DataFrame(test_scaled).head()

,PPR_PRODCD,NPR_PREMIUM,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY
0,9,2418.301728,2,222,45513.429325,1,198,9,99,2
18,0,807.690177,3,555,145010.003453,3,124,2,99,2
23,4,191.160414,3,444,158913.260428,1,251,9,99,2
35,0,401.191521,1,111,45513.429325,3,17,2,99,2
36,4,191.160414,3,333,158913.260428,1,378,12,99,2


In [36]:
target_y.value_counts()

AttributeError: 'numpy.ndarray' object has no attribute 'value_counts'

In [37]:
pd.Series(target_y).head(30)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    1
14    0
15    0
16    0
17    0
18    1
19    0
20    1
21    0
22    1
23    1
24    0
25    0
26    0
27    1
28    0
29    1
dtype: int64

In [38]:
import seaborn as sns
# import matplotlib.pyplot as plt
import matplotlib.pylab as plt
plt.style.use("seaborn")
%matplotlib inline

In [ ]:
# import matplotlib.pyplot as plt

# fig, ax = plt.subplots(figsize=(10,10)) # Sample figsize in inches
# sns.heatmap(train_scaled, annot=True, linewidths=.5, ax=ax)
# plt.title( "HeatMap using Seaborn Method" )
# plt.show()
# # sns.heatmap(train_scaled, annot=True)

In [ ]:
pause

In [39]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

In [40]:
X_train, X_test, y_train, y_test = train_test_split(train_scaled, target_y, test_size=0.25, random_state=0)

In [41]:
pd.DataFrame(X_train).isnull().sum()

PPR_PRODCD         0
NPR_PREMIUM        0
CLF_LIFECD         0
NSP_SUBPROPOSAL    0
NPR_SUMASSURED     0
NLO_TYPE           0
AAG_AGCODE         0
PCL_LOCATCODE      0
OCCUPATION         0
CATEGORY           0
dtype: int64

In [42]:
from sklearn.model_selection import cross_validate,GridSearchCV
from sklearn.metrics import confusion_matrix , accuracy_score
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [43]:
dtree_model = DecisionTreeClassifier(max_depth = 3).fit(X_train, y_train)
gnb = GaussianNB().fit(X_train, y_train)
knn = KNeighborsClassifier(n_neighbors = 22).fit(X_train, y_train)
xgb = XGBClassifier()
# rf = RandomForestClassifier(n_estimators=100)
rf = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=20, oob_score = True) 

param_grid = { 
    'n_estimators': [10, 30],
    'max_features': ['auto', 'sqrt', 'log2']
}

CV_rfc = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 5)
# CV_rfc.fit(X, y)
# print CV_rfc.best_params_

In [44]:
from sklearn.ensemble import VotingClassifier
estimators=[('knn', knn), ('gnb', gnb), ('dtree_model', dtree_model), ('xgb', xgb), ('CV_rfc',CV_rfc)]
ensemble = VotingClassifier(estimators, voting='soft')

In [45]:
ensemble.fit(X_train, y_train)
ensemble.score(X_test, y_test)

[14:20:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.8079740891604436

In [46]:
predictions=ensemble.predict_proba(test_scaled)[:,1]

In [47]:
predictions

array([0.3905953 , 0.3794893 , 0.2293557 , ..., 0.2534319 , 0.20542434,
       0.18446962])

In [48]:
print(predictions.shape)
print(test_scaled.shape)
print(train_scaled.shape)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(12000,)
(12000, 10)
(282815, 10)
(212111, 10)
(70704, 10)
(212111,)
(70704,)


In [ ]:
pause

In [ ]:
# log_loss = log_loss(test_scaled, predictions)
# log_loss

In [ ]:
# def logloss(a, b, eps=1e-15):
#     b = np.clip(b, eps, 1 - eps)
#     return -(a * np.log(b)).sum(axis=1).mean()

In [ ]:
from sklearn.metrics import log_loss
log_loss(test_scaled, predictions)

In [ ]:
second_mrg

In [ ]:
#id_p=_id['Policy ID']


In [ ]:
id_p.shape

In [ ]:
predictions.shape

In [49]:
submit_prep = {'Policy ID':id_p, 'Lapse': predictions}
submission = pd.DataFrame(data = submit_prep)
submission=submission.reset_index(drop=True)
submission.head()

,Policy ID,Lapse
0,PID_VPRO0EG,0.390595
1,PID_O58E151,0.379489
2,PID_FLEBYPG,0.229356
3,PID_KNOH81G,0.360306
4,PID_BEFJS7O,0.221532


In [ ]:
pause

In [50]:
submission.to_csv('Sixth.csv', index=False)